<a href="https://colab.research.google.com/github/biothomme/Linalool/blob/change_research/ncbi_lumberjack_og.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#NCBI Lumberjack
###Uppsala, Dec 2019
This script enables the datamining of the NCBI database for genes in monophyletic groups. Follow the blocks straight forward to gain your result. You can mine x gene entries per Genus/Species of your monophylum and download all sequences in a fasta-file together with 2 informative files:
- <b>..._stat.csv:</b> includes all taxIDs (only on level genus/species) and Accession IDs for your gene within the monophylum.
- <b>..._data.csv:</b> corresponds to your fasta file and includes very important data of the sequences
<br> 
<br> 



#0 Usage
This script works with the help of two powerful instutitions: biopython (https://biopython.org/wiki/Documentation) and through a wonderful backdoor to NCBI databases: the Entrez E-utilities service (https://www.ncbi.nlm.nih.gov/books/NBK25497/). If you use the sequence data for scientific purpose (and not only for fun ;-) ), please cite those sources properly (especially biopython requires a citation of Cock et al., 2009!

#1 `ncbi_miner`

At first biopython needs to be installed.

In [2]:
!pip install biopython

     |████████████████████████████████| 2.3MB 9.5MB/s 


In [0]:
#@title 1.1 The function {display-mode: 'form'}
#@markdown Please execute this field. It contains the function `ncbi_miner`, which is the core of this notebook. 
#@markdown It feeds through the NCBI jungle like the hyperactive caterpillar of a leaf miner.
"""
Created on Tue Nov 26 09:29:22 2019

@author: Thomsn
"""

__author__ = 'thomas m. huber'
__email__ = ['thomas.huber@evobio.eu']

def ncbi_miner(taxon_query,
               gene_name,
               gene_length,
               my_mail,
               outgroups = [],
               coding_sequence = True,
               exclude_query = None,
               my_API_key = None,
               tree_resolution = 'genus',
               length_tolerance_percent = 20,
               upper_tolerance = 2,
               search_limit = 10000,
               entries_per_genus = 1,
               entries_per_tax = 1,
               taxonlist_path = None,
               random_mining = False):
    import os
    import pandas as pd
    from Bio import SeqIO
    from Bio import Entrez
    from datetime import datetime
    from urllib.error import HTTPError
    if random_mining:
        from random import shuffle
    start_time = datetime.now()


    newpath = f'{datetime.now().strftime("%Y_%m_%d")}_{taxon_query}_{gene_name}'
    if not os.path.exists(newpath):
        os.makedirs(newpath)
        print(f'Step 0:\n >> Directory {newpath} was created.')


    gl_lowend = gene_length * (100 - length_tolerance_percent)/100
    gl_highend = gene_length * (100 + upper_tolerance * length_tolerance_percent)/100
    
    tree_resolution = str(tree_resolution).lower()

    tax_columns = ['taxID', 'genus', 'epithet', 'entry_UIDs', 'count', 'outgroup']

    if tree_resolution == 'genus':
        taxon_level = 'Genus'
    elif tree_resolution == 'species':
        taxon_level = 'species'
    else:
        print(f'Mining data with tree resolution on the taxonomic level \'{tree_resolution}\' \
is not possible with this script. Try \'genus\' (default) or \'species\'.')

    if taxonlist_path:
        try:
            old_taxon_list = pd.read_csv(taxonlist_path)
        except FileNotFoundError:
            print(f'It was not possible to find input file {taxonlist_path}, please check \
the path and restart \'ncbi_miner\'.')
            return
        else:
            if all([(any(old_taxon_list.keys() == i )) for i in tax_columns]):
                print(f'The csv-file {taxonlist_path} was loaded. Step 1 will be \
skipped. Follow up steps will use those taxa to search for sequences.')

                Entrez.email = my_mail
                Entrez.api_key = my_API_key
                all_taxaIDs = list(old_taxon_list['taxID'])
        
                progress_criterion = len(all_taxaIDs) // 20
                percentage_factor = 20/100
        
                print(f'Step 2:\n >> Esearch for gene entries of all species in the taxon {taxon_query}. \
This may take time, so keep the internet connection, chill down, drink a tea. \n The progress is ...')
                taxon_list = pd.DataFrame(columns = tax_columns)
                for i, taxon in enumerate(all_taxaIDs):
                    k = i + 1
                    if (k % progress_criterion) == 1:
                        print(f' - {int(i // (progress_criterion*percentage_factor))} % -')
                    else:
                        pass

                    if coding_sequence:
                        query = f'{gene_name}[Gene Name] txid{str(taxon)}'
                    else:
                        query = f'{gene_name} txid{str(taxon)}'
                    if exclude_query:
                        exclude_string = f' NOT {exclude_query}'
                        query = f'{query}{exclude_query}'
                    try:
                        with Entrez.esearch(db='nuccore', term=query, retmax=search_limit, sort='date released') as handle:
                            gene_record = Entrez.read(handle)
                    except HTTPError:
                            print('New Error, but RUN Forrest RUN!')
                    else:
                            count = gene_record['Count']
                            if count == '0':
                                pass
                            else:
                                taxon_list.loc[i] = [taxon,
                                                     old_taxon_list['genus'].iloc[i],
                                                     old_taxon_list['epithet'].iloc[i],
                                                     gene_record['IdList'],
                                                     int(gene_record['Count']),
                                                     old_taxon_list['outgroup'].iloc[i]]
                taxon_list.to_csv(f'{newpath}/{taxon_query}_{gene_name}_stat.csv')
                print(f' >> Done - entry database successfully established. Summary was saved as \
{taxon_query}_{gene_name}_stat.csv')

            ### OLD ###

            else:
                print(f'The input file {taxonlist_path} does not fit with the conditions (header).\
Please change and restart \'ncbi_miner\'.')
                return

    else:
        #### > STEP 1 < ####
    
        Entrez.email = my_mail
        Entrez.api_key = my_API_key

        try:
            with Entrez.esearch(db="taxonomy", term=f'{taxon_query}[orgn]', retmax=search_limit) as handle:
                record = Entrez.read(handle)
        except HTTPError:
            return str('Database error, try later...')
        else:
            all_taxaIDs = record['IdList']
              
                #try:
                 #       with Entrez.esearch(db="taxonomy", term=f'{taxon_query}[orgn]', retmax=search_limit) as handle:
                  #          record = Entrez.read(handle)
                #except HTTPError:
                 #       return str('Database error, try later...')
                #else:
                 #       outgroup_taxaID = pd.Series([record['IdList']])

            print('Step 1:\n >> Done - taxon database successfully established.')
        
    
        #### > STEP 2 < ####
    
        Entrez.email = my_mail
        Entrez.api_key = my_API_key
        
        progress_criterion = len(all_taxaIDs) // 20
        percentage_factor = 20/100
        print(f'Step 2:\n >> Esearch for gene entries of all species in the taxon {taxon_query}. \
This may take time, so keep the internet connection, chill down, drink a tea. \n The progress is ...')
        taxon_list = pd.DataFrame(columns = tax_columns)
        for i, taxon in enumerate(all_taxaIDs, 1):
            if (i % progress_criterion) == 1:
                print(f' - {int(i // (progress_criterion*percentage_factor))} % -')

            try:
                with Entrez.esummary(db="taxonomy", id=taxon, retmax=search_limit) as handle:
                    record = Entrez.read(handle)[0]
            except IndexError:
                print('New Error. Nemas Problemas!')
                break
            except HTTPError:
                print('New Error, but bro, stay seated, we skip it and keep searching...')
            else:
                if record['Rank'] == 'species' and record['Genus'] != '':
                    try:
                        if coding_sequence:
                                query = f'{gene_name}[Gene Name]+txid{str(taxon)}'
                        else:
                                query = f'{gene_name}+txid{str(taxon)}'
                        if exclude_query:
                                exclude_string = f'+NOT+{exclude_query}'
                                query = f'{query}{exclude_query}'
                        with Entrez.esearch(db='nuccore', term=query, retmax=search_limit, sort='pub+date') as handle:
                            gene_record = Entrez.read(handle)
                    except HTTPError:
                        print('New Error! But nothing big to worry about.')
                    else:
                        count = gene_record['Count']
                        if count == '0':
                            pass
                        else:
                            taxon_list.loc[taxon] = [taxon,
                                                 record['Genus'],
                                                 record['Species'],
                                                 gene_record['IdList'],
                                                 int(gene_record['Count']),
                                                 'False']
                else:
                    pass
        og_list = pd.DataFrame(columns = tax_columns)
        if len(outgroups) > 0:
            for og in outgroups:
                taxon = og
                if coding_sequence:
                    query = f'{gene_name}[Gene Name]+{og}[Organism]'
                else:
                    query = f'{gene_name}+{og}[Organism]'
                if exclude_query:
                    exclude_string = f'+NOT+{exclude_query}'
                    query = f'{query}{exclude_query}'
                print(query)
                try:
                    with Entrez.esearch(db='nuccore', term=query, retmax=search_limit, sort='pub+date') as handle:
                          gene_record = Entrez.read(handle)
                    print(f'{gene_record}')
                except HTTPError:
                    print('New Error! But nothing big to worry about.')
                else:
                    count = gene_record['Count']
                    if count == '0':
                        print('There are no entries of the gene for outgroup {og}.')
                        pass
                    else:
                        print(f'{gene_record}')
                        og_list.loc[taxon] = [taxon,
                                            f'{taxon}_gen',
                                            f'{taxon}_sp',
                                            gene_record['IdList'],
                                            int(gene_record['Count']),
                                            'True']

        else:
            print('Warning: You have no outgroups!')
        #for taxon in enumerate(all_taxaIDs, 1):
            taxon_list.to_csv(f'{newpath}/{taxon_query}_{gene_name}_stat_ingroup.csv')
        print(f' >> Done - entry database successfully established. Summary (without outgroups) was saved as \
{taxon_query}_{gene_name}_stat_ingroup.csv')
        
    
    #### > STEP 3 < ####

    print(f'Step 3:\n >> Efetch for each species of the given genera with the most entries \
for the given gene. This may take some time as well, I would answer some mails in the meantime ;-) \n The progress is ...')
    data_list = pd.DataFrame(columns = ['taxID',
                                        'accession',
                                        'length',
                                        'date',
                                        'organism',
                                        'reference',
                                        'gene_information',
                                        'sampling_locality',
                                        'outgroup',
                                        'genus',
                                        'epithet'])
    ### NEW ###
    full_taxon_list = pd.concat([taxon_list, og_list])
    print(og_list)
    if taxon_level == 'Genus':
        genera = full_taxon_list['genus'].unique()
    else:
        genera = full_taxon_list['taxID'].unique()
    print(genera)
    ### OLD ###

    progress_criterion = -( -len(genera) // 10)
    percentage_factor = 10/100
    
    
    for j, genus_ID in enumerate(genera):
        if (j % progress_criterion) == 1:
            print(f' - {int(j // (progress_criterion*percentage_factor))} % -')
        else:
            pass
        if taxon_level == 'Genus':
          all_species = full_taxon_list[full_taxon_list['genus'] == genus_ID]
        else:
          all_species = full_taxon_list[full_taxon_list['taxID'] == genus_ID]
        all_species = all_species.sort_values(by=['count'], ascending=False)
        if len(list(all_species['entry_UIDs'])) < entries_per_genus:
            epg = len(list(all_species['entry_UIDs']))
        else:
            epg = entries_per_genus
        for entry in list(range(epg)):
        ### NEW ###
            entry_list = list(all_species['entry_UIDs'])[entry]
            if random_mining:
                shuffle(entry_list)
        ### OLD ###
            Entrez.email = my_mail
            Entrez.api_key = my_API_key
            entry_counter = 0
            for i, acc_id in enumerate(entry_list):
                try:
                    with Entrez.efetch(db="nuccore", id=acc_id, retmode='text', rettype="gb") as handle:
                        record = SeqIO.read(handle, "genbank")
                except HTTPError:
                    print('New Error, but chill down, everything is soft')
                else:
                    if gl_lowend < len(record) < gl_highend:
                        features = record.features
                        filterframe = [x.type == 'gene' for x in features]
                        if any(filterframe) == True:
                            keyword = 'gene'
                        else:
                            filterframe = [x.type != 'source' for x in features]
                            keyword = 'product'
                        gene_features = [x for i, x in enumerate(features) if filterframe[i]==True]
                        try:
                            gene_info = [(x.qualifiers.get(keyword))[0] for x in gene_features]
                        except TypeError:
                            gene_info = ['']
                        sample_locality = features[0].qualifiers.get('country')
                        try:
                            referenza = record.annotations['references'][0]
                        except TypeError:
                            referenza = ['']
                        selection = full_taxon_list[full_taxon_list['taxID'] == genus_ID]
                        data_list.loc[f'{j+1}_{record.id}'] = [genus_ID,
                                            record.id,
                                            len(record),
                                            record.annotations['date'],
                                            record.annotations['organism'],
                                            referenza,
                                            gene_info,
                                            sample_locality,
                                            selection['outgroup'].iat[0],
                                            selection['genus'].iat[0],
                                            selection['epithet'].iat[0]]
                        with open(f'{newpath}/{taxon_query}_{gene_name}.fasta', 'a') as finalfasta:
                            rawseq = str(record.seq)
                            if taxon_level == 'species':
                                fasta_head = str(record.annotations['organism'])
                                fasta_head = fasta_head.replace(' ', '_').replace('.', '').replace('-', '_')
                            else:
                                fasta_head = genus_ID
                            if entries_per_tax == 1 and entries_per_genus == 1:
                                finalfasta.write(f'>{fasta_head}\n')
                            else:
                                finalfasta.write(f'>{fasta_head}_{record.id}\n')
                            finalfasta.write(f'{rawseq}\n\n')
                ### NEW ###
                        entry_counter += 1
                        if entry_counter == entries_per_tax:
                            break
                        else:
                            pass
                ### OLD ###
                    else:
                        pass
    print('Updating the outgroup data...')
    out_selection = data_list[data_list['outgroup'] == 'True']
    for i, org in enumerate(out_selection['organism']):
        print(org)
        try:
            with Entrez.esearch(db="taxonomy", term=f'{org}[Scientific Name]', retmax=search_limit) as handle:
                record = Entrez.read(handle)
        except HTTPError:
            print(f'Error, did not find {org}.')
        else:
            txid = record['IdList'][0]
            print(txid)
            try:
                with Entrez.esummary(db="taxonomy", id=txid, retmax=search_limit) as handle:
                    spef_record = Entrez.read(handle)[0]
            except IndexError:
                print('New Error. Nemas Problemas!')
                break
            except HTTPError:
                print('Errore furore, no problemo spaghetto!')
            else:
                og_gen = spef_record['Genus']
                og_sp = spef_record['Species']
                data_list.loc[data_list['organism'] == org, ['taxID']] = txid
                data_list.loc[data_list['organism'] == org, ['genus']] = og_gen
                data_list.loc[data_list['organism'] == org, ['epithet']] = og_sp
                taxon_list.loc[txid] = [txid,
                                  og_gen,
                                  og_sp,
                                  og_list['entry_UIDs'][i],
                                  og_list['count'][i],
                                  'False']


    data_list.to_csv(f'{newpath}/{taxon_query}_{gene_name}_data.csv')
    taxon_list.to_csv(f'{newpath}/{taxon_query}_{gene_name}_stat.csv')
    print(f' >> Done - most recent fasta sequences were collected and successfully concatenated. \
It was saved in the file {taxon_query}_{gene_name}.fasta and is proove for nexus conversion. \
Summary of used sequences was saved as {taxon_query}_{gene_name}_data.csv. In addition the \
outgroup was added to the stat-file and saved as: {taxon_query}_{gene_name}_stat.csv.')
    stop_time = datetime.now()
    process_time = stop_time - start_time
    print(f' >> NCBI mining finished. It took {process_time.seconds // 60} min, \
{process_time.seconds % 60} sec. The files are stored in the directory {newpath}.')



#2 Parameters

##2.1 Mandatory arguments for search
Please enter your monophylum as query and run the block.

In [0]:
#{display-mode: 'form'}
#@markdown Name of monophyletic taxon:
taxon_query = 'Caltha' #@param {type:"string"}
#@markdown Name of gene:
gene_name = 'matk' #@param {type:"string"}
#@markdown Estimate of gene length (check at NCBI, pubmed, ...):
gene_length = 750 #@param {type:"number"}
#@markdown Enter your mail adress (mandatory for NCBI search):
my_mail = 'thomas.huber@evobio.eu' #@param {type:"string"}
coding_sequence = True #@param {type:"boolean"}

In [0]:
#{display-mode: 'form'}
outgroups = ['Gentianaceae',  'Campanula'] #@param {type:"raw"}

##2.2 Optional arguments for search
The following parameters can be adjusted. Please run the block afterwards.

In [0]:
#{display-mode: 'form'}
#@markdown Enter your NCBI API key (increases search pace by factor ~ 3). Read more here : https://ncbiinsights.ncbi.nlm.nih.gov/2017/11/02/new-api-keys-for-the-e-utilities/
#@markdown <br/>(!) You will need to set it within quotation marks (e.g. 'AZNE192930N8D...NDJE9D0').
my_API_key =  '7c7333f5a6fbaac7af1a67a8b70edc19e309'#@param {type:"raw"}
#@markdown Which resolution should your tree have (default: Genus)? i.e. taxonomic level of external branches
tree_resolution = 'species' #@param ['genus', 'species']
#@markdown Tolerance (in percent) of gene length for the retrieved sequences setting lower limit (default: 20).
length_tolerance_percent = 100 #@param {type:"slider", min:0, max:100, step:1}
#@markdown Factor for tolerance setting the upper limit to retrieve sequences.
upper_tolerance = 2 #@param {type:"number"}
#@markdown Enter the limit of retrieved results per search (default: 10000).
search_limit = 10000 #@param {type:"slider", min:1, max:10000, step:1}
#@markdown How many species per genus do you want to retrieve? The species within a genus will be ranked after count of entries for the given gene.
entries_per_genus = 1 #@param {type:"slider", min:1, max:20, step:1}
#@markdown How many entries do you want to retrieve per species/taxon?
entries_per_tax = 1 #@param {type:"slider", min:1, max:20, step:1}
#@markdown Usually the results will be sorted by increasing age, so the most recent entries should be retrieved. Do you want to change it to random order?
random_mining = False #@param {type:"boolean"}
taxonlist_path = None

##(2.3 Upload of accession list as csv)
Do you already have a set of selected accession IDs as a result of a previous use of lumberjack (i.e. deforestation) and you want to use it for a new search (e.g. different genes)? Then you should upload it here:

In [0]:
#@title --> Upload here! <-- {display-mode: 'form'}
from google.colab import files
uploaded = files.upload() 

for fn in uploaded.keys():
  taxonlist_path = fn
  print(f'Upload successful, {fn} will be used in the following ncbi_miner session')

#3 Let's go!

In [126]:
#@markdown On the left you can see how the function is called. Run it, to conduct the deforestation of the NCBI jungle.
ncbi_miner(taxon_query = taxon_query,
               gene_name = gene_name,
               gene_length = gene_length,
               my_mail = my_mail,
               outgroups = outgroups,
               coding_sequence = coding_sequence,
               my_API_key = my_API_key, 
               tree_resolution = tree_resolution,
               length_tolerance_percent = length_tolerance_percent,
               upper_tolerance = upper_tolerance,
               search_limit = search_limit,
               entries_per_genus = entries_per_genus,
               entries_per_tax = entries_per_tax,
               taxonlist_path = taxonlist_path,
               random_mining = random_mining)

Step 0:
 >> Directory 2019_12_21_Caltha_matk was created.
Step 1:
 >> Done - taxon database successfully established.
Step 2:
 >> Esearch for gene entries of all species in the taxon Caltha. This may take time, so keep the internet connection, chill down, drink a tea. 
 The progress is ...
matk[Gene Name]+Gentianaceae[Organism]
{'Count': '1227', 'RetMax': '1227', 'RetStart': '0', 'IdList': ['1783492105', '1783489839', '1776852779', '1775178346', '1771904382', '1735237277', '1714006962', '1693286041', '1572523471', '1622822656', '1597478977', '1597477956', '1597477908', '1597477905', '1597477903', '1597477901', '1597477898', '1597477313', '1586082147', '1198275461', '1198275235', '1198275047', '1198274651', '1198274385', '1198274229', '1198274151', '1198274143', '1198273907', '1198273535', '1198273453', '1198273245', '1198273219', '1333443866', '1333443656', '1318751401', '1509837747', '1509837656', '1509837566', '1475997778', '1475997776', '1475997774', '1475997772', '1475997770', '147

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


Campanula calcicola
1241042
 >> Done - most recent fasta sequences were collected and successfully concatenated. It was saved in the file Caltha_matk.fasta and is proove for nexus conversion. Summary of used sequences was saved as Caltha_matk_data.csv. In addition the outgroup was added to the stat-file and saved as: Caltha_matk_stat.csv.
 >> NCBI mining finished. It took 0 min, 26 sec. The files are stored in the directory 2019_12_21_Caltha_matk.


#4 ReSearch with different gene but same accessions
This block uses the data which was previously produced on the server. If you want to upload the file you use, change `gene_name`and `gene_length` in 2.1, upload the file in 2.3 and finally run `ncbi_miner` in 3.

In [0]:
#{display-mode: 'form'}
#@markdown What is the name of your new gene?
new_gene_name = 'rbcl' #@param {type:"string"}
#@markdown Estimate the length of your new gene:
new_gene_length = 750 #@param {type:"number"}
#@markdown When did you perform the search for accessions for the old gene?
date_of_old_search = "2019-12-12"#@param {type:"date"}
date_underscore = date_of_old_search.replace('-', '_')

old_file = f'{date_underscore}_{taxon_query}_{gene_name}/{taxon_query}_{gene_name}_stat.csv'

In [0]:
#{display-mode: 'form'}
#@markdown Run the search with new gene - old accessions (same parameters).
ncbi_miner(taxon_query = taxon_query,
               gene_name = new_gene_name,
               gene_length = new_gene_length,
               my_mail = my_mail,
               my_API_key = my_API_key,
               tree_resolution = tree_resolution,
               length_tolerance_percent = length_tolerance_percent,
               upper_tolerance = upper_tolerance,
               search_limit = search_limit,
               entries_per_genus = entries_per_genus,
               entries_per_tax = entries_per_tax,
               taxonlist_path = old_file,
               random_mining = random_mining)
#@markdown ---


#5 Download
After producing your data, you can download it all together. Date will be assumed to be today.

In [0]:
#@title 5.1 Zip file for (Re)Search* {display-mode: 'form'}
from datetime import datetime
import shutil
from google.colab import files
#@markdown When did you perform the (re)search you want to download?
date_of_search = "2019-12-21"#@param {type:"date"}
date_uscore = date_of_search.replace('-', '_')
#@markdown Search (3.) or  ReSearch (4.)?
option = 'search' #@param ['search', 'research']

if option == 'search':
  zip_gene = gene_name
else:
  zip_gene = new_gene_name
directory = f'{date_uscore}_{taxon_query}_{zip_gene}'
folder = shutil.make_archive(directory, 'zip', directory)
files.download(folder)


#@markdown <br><br>The data you produced will be stored into a ZIP file and downloaded. <br> <br> *) Search and ReSearch store their files in different folders. Those would need to be downloaded subsequently.



#References
<i>Cock PA, Antao T, Chang JT, Chapman BA, Cox CJ, Dalke A, Friedberg I, Hamelryck T, Kauff F, Wilczynski B and de Hoon MJL (2009):</i> Biopython - freely available Python tools for computational molecular biology and bioinformatics. Bioinformatics, 25, 1422-1423

---
Thank you for using the NCBI lumberjack, have fun with the sequences!<br>
`Questions: thomas.huber@evobio.eu`